In [ ]:
!pip install -U peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 28.6 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.7.0
    Uninstalling peft-0.7.0:
      Successfully uninstalled peft-0.7.0


In [ ]:
# import librairies
from datasets import load_dataset
from diffusers import StableDiffusionPipeline

## Load datasets

In [ ]:
#!git clone https://huggingface.co/datasets/Loulblemo/diffusion_skyscrapers_city_building

In [ ]:
# import pandas as pd
# df = pd.read_csv('/content/diffusion_skyscrapers_city_building/metadata.csv')
# df

In [ ]:
# import matplotlib.pyplot as plt
# from PIL import Image

# img = Image.open('/content/diffusion_skyscrapers_city_building/data/2024_05_03_204949.png')
# plt.imshow(img)
# plt.show()


## Load lora script

In [ ]:
# need to install diffusers to use lora script

# !pip install git+https://github.com/huggingface/diffusers
# !pip install -r https://raw.githubusercontent.com/huggingface/diffusers/main/examples/text_to_image/requirements.txt


In [ ]:
!pip install accelerate

In [ ]:
!accelerate config default
# accelerate configuration saved at $HOME/.cache/huggingface/accelerate/default_config.yaml

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
import wandb
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler, AutoPipelineForText2Image
from huggingface_hub import model_info

In [ ]:
!wget -q https://raw.githubusercontent.com/huggingface/diffusers/main/examples/text_to_image/train_text_to_image_lora.py

## Training a LoRA with Diffusers Library

In [ ]:
# Set environment variables for use in shell commands
%env MODEL_NAME=runwayml/stable-diffusion-v1-5
%env OUTPUT_DIR=./finetune_lora/pokemon
%env HUB_MODEL_ID=pokemon-lora
%env DATASET_NAME=svjack/pokemon-blip-captions-en-zh

# Create output directory
!mkdir -p $OUTPUT_DIR

# Launch training with accelerate
!accelerate launch --mixed_precision="bf16" train_text_to_image_lora.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --dataset_name=$DATASET_NAME \
  --dataloader_num_workers=8 \
  --resolution=512 \
  --center_crop \
  --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=15 \
  --learning_rate=1e-04 \
  --max_grad_norm=1 \
  --lr_scheduler="cosine" \
  --lr_warmup_steps=0 \
  --output_dir=$OUTPUT_DIR \
  --checkpointing_steps=5 \
  --caption_column="en_text" \
  --validation_prompt="A pokemon with blue eyes." \
  --seed=1337


## Using Your Trained LoRA

In [ ]:
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from huggingface_hub import model_info
import torch

# LoRA weights ~3 MB
model_path = "pcuenq/pokemon-lora"

info = model_info(model_path)
model_base = info.cardData["base_model"]
pipe = StableDiffusionPipeline.from_pretrained(model_base, torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

pipe.unet.load_attn_procs(model_path)
pipe.to("cuda")

image = pipe("Green pokemon with menacing face", num_inference_steps=25).images[0]
image.save("green_pokemon.png")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

ValueError: `low_cpu_mem_usage=True` is not compatible with this `peft` version. Please update it with `pip install -U peft`.

In [ ]:

from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained("runwayml/stable-diffusion-v1-5",
                                                     torch_dtype=torch.float16
                                                    ).to("cuda")
pipeline.load_lora_weights("finetune_lora/pokemon",
                           weight_name="pytorch_lora_weights.safetensors")
image = pipeline("A pokemon with blue eyes").images[0]